# Welcome to PYNQ

## Getting Started

To get started using PYNQ, try running the example notebooks in the folders described below. 

* **getting_started**: includes an introduction to using Jupyter notebook with PYNQ, the Python environment, and how to use some basic features of the curernt platform. 

* **common**: contains example notebooks on how to download an overlay, how to set the Zynq clocks, how to execute Linux shell commands, and how to use USB devices.

If other overlays or packages are installed, other folders with example notebooks may also be available in this directory.  


## Documentation

Please see the latest <a href="http://pynq.readthedocs.io">PYNQ Documentation on readthedocs</a>.  


## Support

For questions or support, go to the forum on the <a href="http://www.pynq.io">PYNQ project webpage </a>.


## Project webpage

You can find details on the <a href="http://www.pynq.io">PYNQ project webpage </a>.


## GitHub

The PYNQ Repository is hosted on github: <a href="https://github.com/Xilinx/PYNQ">PYNQ GitHub Repository</a>.

In [2]:
import os

In [3]:
from pynq import Overlay

In [4]:
os.getcwd()

'/home/xilinx/jupyter_notebooks'

In [5]:
# Use bitstream for PL 
overlay = Overlay(os.getcwd() + "/fifo_examples/fifo_no_interrupt_read_size/fifo.bit")

In [6]:
overlay?

In [9]:
# 1D FIFO
fifo_ip = overlay.fifo
dma = overlay.fifo_dma
fifo_ip?

In [18]:
# View Registers
fifo_ip.register_map


RegisterMap {
  CTRL = Register(AP_START=1, AP_DONE=0, AP_IDLE=0, AP_READY=0, RESERVED_1=0, AUTO_RESTART=0, RESERVED_2=0),
  GIER = Register(Enable=0, RESERVED=0),
  IP_IER = Register(CHAN0_INT_EN=0, CHAN1_INT_EN=0, RESERVED=0),
  IP_ISR = Register(CHAN0_INT_ST=0, CHAN1_INT_ST=0, RESERVED=0),
  size = Register(size=10),
  size_ctrl = Register(size_ap_vld=0, RESERVED=0)
}

In [ ]:
help(fifo_ip.register_map.CTRL)

In [19]:
import numpy as np
from pynq import allocate

In [20]:
# Will run for one intake
# Enable AUTO_RESTART for continuous runs
fifo_ip.register_map.CTRL.AP_START = 1

In [21]:
# Read numpy array size
fifo_ip.register_map.size

Register(size=10)

In [22]:
# Prove size is read-only
fifo_ip.register_map.size = 1000

AttributeError: can't set attribute

In [23]:
size = int(fifo_ip.register_map.size)
print(size)

10


In [24]:
# size = 10
inStream = allocate(shape=(size,), dtype=np.int32)
outStream = allocate(shape=(size,), dtype=np.int32)

for i in range(size):
    inStream[i] = 100
    
inStream.flush()
print(inStream)
print(outStream)


[100 100 100 100 100 100 100 100 100 100]
[0 0 0 0 0 0 0 0 0 0]


In [25]:
dma.sendchannel.transfer(inStream)
print("Completed transfer inStream")



Completed transfer inStream


In [26]:
dma.recvchannel.transfer(outStream)
print("Completed transfer outStream")

Completed transfer outStream


In [27]:
# Call if TLAST is not set high in HLS code
dma.sendchannel.wait()
print("Completed wait send")


Completed wait send


In [28]:
# Call if TLAST is not set high in HLS code
dma.recvchannel.wait()
print("Completed wait recv")

Completed wait recv


In [29]:
print(outStream)

[100 100 100 100 100 100 100 100 100 100]


In [30]:
inStream.freebuffer()
print("Freed inStream buffer")

Freed inStream buffer


In [31]:
outStream.freebuffer()
print("Freed outStream buffer")

Freed outStream buffer


In [32]:
fifo_ip.register_map

RegisterMap {
  CTRL = Register(AP_START=0, AP_DONE=0, AP_IDLE=1, AP_READY=0, RESERVED_1=0, AUTO_RESTART=0, RESERVED_2=0),
  GIER = Register(Enable=0, RESERVED=0),
  IP_IER = Register(CHAN0_INT_EN=0, CHAN1_INT_EN=0, RESERVED=0),
  IP_ISR = Register(CHAN0_INT_ST=0, CHAN1_INT_ST=0, RESERVED=0),
  size = Register(size=10),
  size_ctrl = Register(size_ap_vld=0, RESERVED=0)
}